# How to Price Your Home
## A data exploration, modeling and evaluation using the Ames, Iowa housing sales dataset

### Overview

- Initial exploration
    - Exploration of the target variable
- Data preparation
- Modeling
    - Model-less baseline
    - Multiple linear regression
    - Ridge regression
- Discussion of next steps


Selling a home can be a torturous process. Homeowners rely on real estate agent's opinions and experience to guide them to the right decisions. So many questions haunt the homeowner. Are you pricing it right for the market? Are you pricing it to move quickly or get the maximum price?

My project aims to predict the sale price of houses using the [Ames, Iowa housing dataset](http://jse.amstat.org/v19n3/decock.pdf) produced by Dean De Cook, of Truman University in 2011. The training dataset used for this anaylsis was sourced from the [Advanced Regression Techniques Kaggle competition](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview). 

I began with a baseline that predicts only the mean house price from the training data for each house, but this model does very poorly (of course). My final model is a Ridge regression model, which had an R2 Score of 0.78 and a MAE of $24,244.

### Data Description

The subset of the Ames, Iowa dataset includes 1,460 home sales and record 81 variables for each home. The variables describe the details of the home, its ammenities, the lot, the zoning, and descriptors of the type of sale itself. While many variables are not within the control of the homeowner, such as lot size or zoning, including them in the eventual model will be useful for controlling for the impact. 

#### Data Descisions

**Removed Observations**

I used the variable `SaleType`, to exclude new construction and estate trasfers from the analysis. I assumed that homeowners most interested in this analysis would be currently living _in_ their home and looking to sell. I then dropped the variable. Removing those home sales reduced the dataset to 1,295 observations.



**Removed Variables**

- `Id` was removed, as it is a house identifier and not descriptive. 
- `FireplaceQu` was removed because the majority of houes do not have fireplaces. 
- `MiscFeature` was removed because there were so few misc features in the dataset. Sheds, second garages, and such counted as misc features.
- All variables that assessed the quality or condition of a _specific part of the house_ were dropped. Only Overall Condition and Overall Quality were retained. I mad this choice because the Overall variables are composits of the individual. If housing condition proves to be an important variable, later analysis can examine how the quality of individual rooms interact with overall quality and impact the final sale. 

**Adjusted Variables**

`Alley`: Only 91 homes in the dataset were listed to have `Alley`s behind the house, and the values for that variable were "gravel", "paved", or missing. I created a new variable to just capture **if** the property was adjacent to the property and drop the original alley type variable.

`PoolArea` & `PoolQC`: There are only 6 pools in the dataset. I dropped `PoolQC` and transformed `PoolArea` into `HasPool`

`HasFireplace`: reduced `Fireplaces`, which is a count of fireplaces in the home, to a binary yes/no if the home has a fireplace at all.

`HasFence`: reduced `Fence`, a list of fence condition descriptors, to a binary yes/no if the home came with a fence.

`LotFrontage`: filled the missing values with 0. The variable measures the number of feet of street the property touches. 

`GarageFinish`: filled missing values with `Unf`, "unfinished", as those are the properties that are missing garages. An interaction term with `HasGarage`, and setting `Unf` as the baseline when the variable is one hot encoded, will account for any conflicts. 

**New Variables**

`HasGarage`: if `GarageArea` equals zero, then the property has no garage. It also explains why some observations have missing variables for garage attributes. Upon investigation, those observations have zero garage area. 

`GarageAreaPerCar`: `GarageArea` divided by `GarageCars`. With this created I could then drop `GarageArea`. It's assumed that the more cars, the larger the garage, so I did not want both variables in there violating regression assumptions. Having the ratio will control for multi-car garages that actually have very little space.

A [full description of the variables](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data?select=data_description.txt) in the dataset is available. 

In [1]:
# Imports
import pandas as pd
import numpy as np

# Custom code
import code.data_prep_code as dpc

# Visalization libraries
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Data Preparation and Modeling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

# Evaluation
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

# Feature importance library
import eli5
from eli5.sklearn import PermutationImportance

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [2]:
df = pd.read_csv('data/train.csv')

In [3]:
df = dpc.address_nas(df)

In [4]:
na_count = df.isna().sum().sort_values(ascending=False)
na_count[na_count>0]

GarageFinish    81
GarageType      81
GarageYrBlt     81
BsmtFinType2    38
BsmtExposure    38
BsmtFinType1    37
MasVnrType       8
MasVnrArea       8
Electrical       1
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 68 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 non-null object
MasVnrType       1452 non-null object
MasVnrArea       1452 non

In [ ]:
df.head()

In [ ]:
# Exploring nulls and data types
df.info()

In [11]:
df.GarageType.value_counts()

Attchd     870
Detchd     387
BuiltIn     88
Basment     19
CarPort      9
2Types       6
Name: GarageType, dtype: int64

In [5]:
df.GarageFinish.value_counts()

Unf    605
RFn    422
Fin    352
Name: GarageFinish, dtype: int64

In [ ]:
df_used_homes = df.loc[~df.SaleType.str.contains("New|COD")].copy()
df_used_homes.drop(columns = ['SaleType'], inplace = True)
df_used_homes.shape

In [ ]:
df_used_homes['HasAlley'] = 0
df_used_homes.Alley.fillna("not",inplace=True)
df_used_homes.loc[(df_used_homes.Alley.str.contains("Grvl|Pave")), 'HasAlley'] = 1
df_used_homes.drop(columns = ['Alley'], inplace=True)


In [ ]:
df_used_homes['HasPool'] = 0
df_used_homes.loc[(df_used_homes.PoolArea > 0), 'HasPool'] = 1
df_used_homes.drop(columns = ['PoolArea', 'PoolQC'], inplace=True)

In [ ]:
df_used_homes['HasFirePlace'] = 0
df_used_homes.loc[(df_used_homes.Fireplaces > 0), 'HasFirePlace'] = 1
df_used_homes.drop(columns = ['Fireplaces', 'FireplaceQu'], inplace=True)

In [ ]:
df_used_homes['HasFence'] = 0
df_used_homes.loc[(~df_used_homes.Fence.isna()), 'HasFence'] = 1
df_used_homes.drop(columns = ['Fence'], inplace=True)

In [ ]:
df_used_homes.drop(columns=["MiscFeature"], inplace = True)

In [ ]:
df_used_homes.LotFrontage.fillna(0,inplace=True)

In [ ]:
test = df_used_homes.columns.tolist()

In [ ]:
redundant = [k for k in test if ('Qual' in k or 'Cond' in k) and ("Overall" not in k) and (k!= "SaleCondition")]

In [ ]:
df_used_homes.drop(columns = redundant, inplace = True )

In [ ]:
na_count = df_used_homes.isna().sum().sort_values(ascending=False)
na_count[na_count>0]

In [ ]:
# Evaluating the breakdown of each numeric column
df.describe()

#### Explore Target Variable

The target variable shows that there are some outliers in the data, which are homes that were sold at much higher prices than most of the other homes in the dataset. I kept these outliers in, however may consider doing a transformation at some point to normalize the target variable. Alternatively, I could train my model without those outliers, with the understanding that then my model would only be accurate at homes that would be sold at prices less than a certain amount.

In [ ]:
# plotting the distribution of the target variable with a boxplot
plt.figure(figsize=(10,8))
sns.boxplot(x=df['SalePrice'])
plt.title("Boxplot for Target Variable")
plt.xlabel("Price in Dollars")
plt.show()

In [ ]:
# plotting the correlation between variables and the target using a heatmap
plt.figure(figsize=(2,10))
sns.heatmap(df.corr()[['SalePrice']])
plt.title("Correlation of Each Independent Variable \n with the Target Variable \n")
plt.show()

The most correlated variable with the target is `OverallQual`, which captures the ranking of the Overall Quality of a home

In [ ]:
test = df.groupby('OverallQual')['SalePrice'].mean()

In [ ]:
sns.scatterplot(test, test.index)

### Data Preparation



In [ ]:
# Using only numeric columns
num_cols = list(df.dtypes[df.dtypes != 'object'].index)

In [ ]:
# subsetting my dataframe
df_num = df[num_cols]
df_num.head()

In [ ]:
def summarize_dataframe(df):
    '''
    Summarizes each column in a Pandas dataframe, where each row of the 
    summary output is a column of the input dataframe, df
    Will show the datatype of data in the column, the number of missing values
    in that column, and the number of unique values in the column
    -
    Input:
    df : Pandas dataframe
    -
    Output:
    summary : Pandas dataframe, now showing column details
    '''
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index'] # name of each variable 
    summary = summary[['Name','dtypes']] # data type of each variable
    summary['Missing'] = df.isnull().sum().values # number of missing values  
    summary['Uniques'] = df.nunique().values # number of unique values
    return summary

In [ ]:
summarize_dataframe(df_num)

In [ ]:
# Dropping the three columns with null values: 
# LotFrontage, MasVnrArea and GarageYrBlt
df_num.dropna(axis=1, inplace=True)

In [ ]:
# Splitting my data into X and Y
target = 'SalePrice'
used_cols = list(col for col in df_num.columns.to_list() if col not in [target, 'Id'])

X = df_num[used_cols]
y = df_num[target]

# Creating a split in my data for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, 
                                                    random_state=42)

In [ ]:
# Scaling my data
scaler = RobustScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Modeling



#### Model-Less Baseline

I began with a baseline that predicts only the mean house price from the training data for each house, but this model does very poorly (of course). My final model is a Ridge regression model, which had an R2 Score of 0.78 and a MAE of $24,244.

I began with a model-less baseline which simply predicts the average sale price from the training set.

In [ ]:
# Grabbing the target mean from the training data
y_mean_train = y_train.mean()

# Creating predictions for the train and test sets
baseline_y_pred_train = ([y_mean_train]*len(y_train))
baseline_y_pred_test = ([y_mean_train]*len(y_test))

In [ ]:
# Evaluating my model-less baseline
# Checking the scores on both train and test set to look for overfitting
print(f"Training R2: {r2_score(y_train, baseline_y_pred_train)}")
print(f"Testing R2: {r2_score(y_test, baseline_y_pred_test)}")
print("---")
print(f"Training MAE: {mean_absolute_error(y_train, baseline_y_pred_train)}")
print(f"Testing MAE: {mean_absolute_error(y_test, baseline_y_pred_test)}")

#### Linear Regression Model

I then used a multiple linear regression model on my 33 numeric features that I already prepared and scaled.

In [ ]:
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

In [ ]:
y_pred_train = lr.predict(X_train_scaled)
y_pred_test = lr.predict(X_test_scaled)

In [ ]:
print(f"Training R2: {r2_score(y_train, y_pred_train)}")
print(f"Testing R2: {r2_score(y_test, y_pred_test)}")
print("---")
print(f"Training MAE: {mean_absolute_error(y_train, y_pred_train)}")
print(f"Testing MAE: {mean_absolute_error(y_test, y_pred_test)}")

#### Ridge Regression Model

Looking at the correlation between variables in the test set, I can see that there is correlation between features. I thus use a Ridge regression model to attempt to regularize away from the effects of that multicollinearity.

In [ ]:
# plotting the correlation between features using a heatmap
plt.figure(figsize=(10,10))
sns.heatmap(df_num[used_cols].corr())
plt.title("Exploring Correlation Between Features")
plt.show()

In [ ]:
ridge = Ridge(random_state=42)
ridge.fit(X_train_scaled, y_train)

ridge_y_pred_train = ridge.predict(X_train_scaled)
ridge_y_pred_test = ridge.predict(X_test_scaled)

In [ ]:
print(f"Training R2: {r2_score(y_train, ridge_y_pred_train)}")
print(f"Testing R2: {r2_score(y_test, ridge_y_pred_test)}")
print("---")
print(f"Training MAE: {mean_absolute_error(y_train, ridge_y_pred_train)}")
print(f"Testing MAE: {mean_absolute_error(y_test, ridge_y_pred_test)}")

#### Checking Feature Importance

I'm using the eli5 library to do permutation importance to explore which feature was the most important to my Ridge regression model.

In [ ]:
# printing the weights, aka importance, for each variable 
perm = PermutationImportance(ridge, random_state=1).fit(X_test_scaled, y_test)
eli5.show_weights(perm, feature_names = X_train.columns.tolist(), top=None)

Unsurprisingly, the most important feature to my Ridge regression model is the one which is most correlated with the target: `OverallQual`

### Next Steps

In the future I would like to first use more of the categorical features, and perhaps encode some of the discrete features I used in my final model. I would also like to then only use the most important features, perhaps by regularizing using both LASSO and Ridge through an ElasticNet model. I could also only use the top 5-10 features based on Permutation Importance. 

I'd also like to explore capping my training data so that I create a model that only works on houses priced up to a certain point. This would narrow the target audience of my model, but at the same time Could make it work better on houses that are priced within a specific range. Thus, if I received a result from my model outside that range I could know that my model is likely not accurately pricing that home and thus should be handled using different techniques or models. 